Этап 1. Исследовательский анализ (EDA)

In [ ]:
import os, re, json, random, pathlib
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

BASE_DIR   = Path("data/nutrition/data")
IMG_DIR    = BASE_DIR / "images"
DISH_CSV   = BASE_DIR / "dish.csv"
INGR_CSV   = BASE_DIR / "ingredients.csv"

for p in [BASE_DIR, IMG_DIR, DISH_CSV, INGR_CSV]:
    print(p, "| exists:", p.exists())

In [ ]:
dish = pd.read_csv(DISH_CSV)
ingr = pd.read_csv(INGR_CSV)

print("dish shape:", dish.shape)
print("ingredients shape:", ingr.shape)

display(dish.head(3))
display(ingr.head(3))

In [ ]:
split_counts = dish["split"].value_counts()
print("split counts:\n", split_counts)

In [ ]:
# Проверим, чтобы один и тот же dish_id не встречался одновременно в train и в test
leak = dish.groupby("dish_id")["split"].nunique()
leak_dishes = leak[leak>1]
print("дубликаты dish_id в разных сплитах:", int((leak_dishes>0).sum()))

In [ ]:
# Формируем путь к изображению
dish["image_path"] = dish["dish_id"].apply(lambda d: str(IMG_DIR / str(d) / "rgb.png"))

# Возьмём 6 изображений
sample = dish.sample(6, random_state=42)

# Отображаем изображения
plt.figure(figsize=(15, 6))
for i, (_, row) in enumerate(sample.iterrows(), 1):
    img = Image.open(row["image_path"]).convert("RGB")
    plt.subplot(2, 3, i)
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"dish_id: {row['dish_id']}\n{row['total_calories']} kcal, {row['split']}")
plt.tight_layout()
plt.show()

In [ ]:
def parse_ingr_ids(s: str):
    # 'ingr_0000000122;ingr_0000000026;...' -> ['0000000122','0000000026', ...]
    if pd.isna(s) or not isinstance(s, str) or s.strip()=="":
        return []
    ids = [t.split("_")[-1] for t in s.split(";") if t]
    return ids

id2name = dict(zip(ingr["id"].astype(str).str.zfill(10), ingr["ingr"].astype(str)))

dish["ingr_ids"]  = dish["ingredients"].apply(parse_ingr_ids)
dish["ingr_names"] = dish["ingr_ids"].apply(lambda lst: [id2name.get(x, f"unknown_{x}") for x in lst])

In [ ]:
from collections import Counter
all_ingr = [n for lst in dish["ingr_names"] for n in lst]
cnt = Counter(all_ingr)
top = pd.DataFrame(cnt.most_common(20), columns=["ingredient","count"])
display(top)

plt.figure(figsize=(8,5))
plt.barh(top["ingredient"][::-1], top["count"][::-1])
plt.title("Top-20 ingredients")
plt.tight_layout(); plt.show()

Что предсказываем:

Цель: total_calories (ккал) для каждого dish_id.

Формат задачи: регрессия.

Мультимодальная регрессия (изображение + текст ингредиентов + масса):



*   Image encoder из timm.
*   Text encoder — bert-base-uncased из transformers. На вход — строка с ингредиентами.

Метрика, по которой будем смотреть качество обучения модели - MAE. Оптимизатор: AdamW, разный LR для текстовой и визуальной частей.

Аугментацию можно сделать по минимуму: масштабировать изображения, сделать Affine (имитирует небольшие сдвиги), ColorJitter(имитирует вариации освещения камеры), небольшой CoarseDropout (drop маленьких областей) — повышает устойчивость к локальным артефактам. 

Так как это фото блюд, здесь важны детали. Также фотографии сняты с одного и того же ракурса. 


Наблюдения по EDA:
В датасете 555 ингредиентов, 3262 блюда. В тренировочной выборке - 2755 блюд, в тестовой - 507.
Самые популярные инредиенты - это оливковое масло, соль и чеснок.

In [ ]:
class Config:
    SEED = 42

    # База данных
    BASE_DIR = "/content/nutrition/data/"

    # Модели
    TEXT_MODEL_NAME  = "huawei-noah/TinyBERT_General_4L_312D"
    IMAGE_MODEL_NAME = "tf_efficientnet_b0"

    # Разморозка слоёв
    TEXT_MODEL_UNFREEZE  = "encoder.layer.11|pooler"
    IMAGE_MODEL_UNFREEZE = "blocks.6|conv_head|bn2"

    # Гиперпараметры
    BATCH_SIZE = 256
    EPOCHS = 140
    USE_AMP = True
    NUM_WORKERS = 2

    # LR и регуляризация
    TEXT_LR  = 3e-5
    IMAGE_LR = 1e-4
    HEAD_LR  = 1e-3
    WEIGHT_DECAY = 1e-4

    # Головы
    HIDDEN_DIM  = 256
    MASS_HIDDEN = 32
    DROPOUT = 0.2

    # Сохранение
    SAVE_PATH = "/data/nutrition/data/best_model.pth"

import torch
from scripts.utils import train

device = "cuda" if torch.cuda.is_available() else "cpu"
cfg = Config()

train(cfg, device)

In [ ]:
def train_resume(config, device, save_path):
    seed_everything(config.SEED)
    
    model = MultimodalRegressor(config).to(device)
    
    # Загружаем сохранённые веса
    print(f"Загружаем веса: {save_path}")
    checkpoint = torch.load(save_path)
    model.load_state_dict(checkpoint)
    
    # Замораживаем часть слоёв чтобы уменьшить переобучение
    for name, param in model.named_parameters():
        if 'text_model' in name and 'encoder.layer.0' in name:
            param.requires_grad = False
        if 'image_model' in name and 'blocks.0' in name:
            param.requires_grad = False
    
    # Оптимизатор с уменьшенным LR
    optimizer = AdamW(
        model.parameters(), 
        lr=config.HEAD_LR, 
        weight_decay=config.WEIGHT_DECAY  # Увеличили регуляризацию
    )
    
    tr_tfms = get_transforms(config, split="train")
    te_tfms = get_transforms(config, split="test")
    
    train_ds = CaloriesDataset(config, tr_tfms, split="train")
    val_ds = CaloriesDataset(config, te_tfms, split="test")
    
    train_loader = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True, 
                             num_workers=config.NUM_WORKERS, collate_fn=collate_fn)
    val_loader = DataLoader(val_ds, batch_size=config.BATCH_SIZE, shuffle=False,
                           num_workers=config.NUM_WORKERS, collate_fn=collate_fn)
    
    criterion = nn.L1Loss()
    scaler = torch.cuda.amp.GradScaler(enabled=config.USE_AMP)
    
    best_mae = 59.95
    start_epoch = 141  # Продолжаем с этой эпохи
    additional_epochs = ContinueConfig.EPOCHS  # Дополнительные эпохи
    
    print(f"Продолжаем обучение с эпохи {start_epoch}...")
    
    for epoch in range(start_epoch, start_epoch + additional_epochs):
        model.train()
        total_mae = 0
        batches = 0
        
        for batch in train_loader:
            optimizer.zero_grad()
            
            with torch.cuda.amp.autocast(enabled=config.USE_AMP):
                inputs = {
                    "input_ids": batch["input_ids"].to(device),
                    "attention_mask": batch["attention_mask"].to(device),
                    "image": batch["image"].to(device),
                    "mass": batch["mass"].to(device),
                }
                target = batch["target"].to(device)
                pred = model(**inputs)
                loss = torch.nn.functional.l1_loss(pred, target)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            total_mae += torch.abs(pred - target).sum().item()
            batches += target.size(0)
        
        train_mae = total_mae / batches
        val_mae = validate(model, val_loader, device)
        
        print(f"Epoch {epoch:03d} | Train MAE: {train_mae:.2f} | Val MAE: {val_mae:.2f}")
        
        if val_mae < best_mae:
            best_mae = val_mae
            new_save_path = f"/data/nutrition/data/continued_best_model.pth"
            torch.save(model.state_dict(), new_save_path)
            print(f"New best, saved to {new_save_path} (MAE={best_mae:.2f})")
            
            if best_mae < 50:
                print("Цель достигнута! MAE < 50")
                break
    
    return best_mae

In [ ]:
class ContinueConfig:
    SEED = 42

    # База данных
    BASE_DIR = "/content/nutrition/data/"

    TEXT_MODEL_NAME  = "huawei-noah/TinyBERT_General_4L_312D"
    IMAGE_MODEL_NAME = "tf_efficientnet_b0"

    # Разморозка слоёв - размораживаем всё для тонкой настройки
    TEXT_MODEL_UNFREEZE  = ""
    IMAGE_MODEL_UNFREEZE = ""

    # Гиперпараметры
    BATCH_SIZE = 256
    EPOCHS = 60  # 60 дополнительных эпох (до 200 всего)
    USE_AMP = True
    NUM_WORKERS = 2

    # LR и регуляризация - уменьшаем В 3-5 РАЗ для тонкой настройки
    TEXT_LR  = 1e-5
    IMAGE_LR = 3e-5 
    HEAD_LR  = 1e-4
    WEIGHT_DECAY = 1e-4

    HIDDEN_DIM  = 256
    MASS_HIDDEN = 32
    DROPOUT = 0.4

    SAVE_PATH = "/data/nutrition/data/continued_model.pth"

In [ ]:
save_path = "/data/nutrition/data/best_model.pth"

cfg = ContinueConfig()
device = "cuda" if torch.cuda.is_available() else "cpu"

final_mae = train_resume(cfg, device, save_path)
print(f"Финальный результат после продолжения: {final_mae:.2f}")

In [ ]:
def train_resume_with_scheduler(config, device, save_path):
    seed_everything(config.SEED)
    
    model = MultimodalRegressor(config).to(device)
    checkpoint = torch.load(save_path)
    model.load_state_dict(checkpoint)
    
    # Заморозка (как была)
    for name, param in model.named_parameters():
        if 'text_model' in name and 'encoder.layer.0' in name:
            param.requires_grad = False
        if 'image_model' in name and 'blocks.0' in name:
            param.requires_grad = False
    
    optimizer = AdamW(model.parameters(), lr=config.HEAD_LR, weight_decay=config.WEIGHT_DECAY)
    
    # ДОБАВЛЯЕМ ШЕДУЛЕР
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=8
    )
    
    tr_tfms = get_transforms(config, split="train")
    te_tfms = get_transforms(config, split="test")
    
    train_ds = CaloriesDataset(config, tr_tfms, split="train")
    val_ds = CaloriesDataset(config, te_tfms, split="test")
    
    train_loader = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True, 
                             num_workers=config.NUM_WORKERS, collate_fn=collate_fn)
    val_loader = DataLoader(val_ds, batch_size=config.BATCH_SIZE, shuffle=False,
                           num_workers=config.NUM_WORKERS, collate_fn=collate_fn)
    
    criterion = nn.L1Loss()
    scaler = torch.cuda.amp.GradScaler(enabled=config.USE_AMP)
    
    best_mae = 52.54  # Текущий лучший
    start_epoch = 200  # Продолжаем с этой эпохи
    additional_epochs = 50
    
    print(f"Продолжаем с MAE=52.54, добавляем шедулер")
    
    for epoch in range(start_epoch, start_epoch + additional_epochs):
        model.train()
        total_mae = 0
        batches = 0
        
        for batch in train_loader:
            optimizer.zero_grad()
            
            with torch.cuda.amp.autocast(enabled=config.USE_AMP):
                inputs = {
                    "input_ids": batch["input_ids"].to(device),
                    "attention_mask": batch["attention_mask"].to(device),
                    "image": batch["image"].to(device),
                    "mass": batch["mass"].to(device),
                }
                target = batch["target"].to(device)
                pred = model(**inputs)
                loss = torch.nn.functional.l1_loss(pred, target)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            total_mae += torch.abs(pred - target).sum().item()
            batches += target.size(0)
        
        train_mae = total_mae / batches
        val_mae = validate(model, val_loader, device)
        
        # обновляем шедулер каждую эпоху
        scheduler.step(val_mae)
        
        current_lr = optimizer.param_groups[0]['lr']
        print(f"Epoch {epoch:03d} | Train MAE: {train_mae:.2f} | Val MAE: {val_mae:.2f} | LR: {current_lr:.2e}")
        
        if val_mae < best_mae:
            best_mae = val_mae
            new_save_path = f"/data/nutrition/data/continued_best_model.pth"
            torch.save(model.state_dict(), new_save_path)
            print(f"New best, saved to {new_save_path} (MAE={best_mae:.2f})")
            
            if best_mae < 50:
                print("Цель достигнута! MAE < 50")
                break
    
    return best_mae

In [ ]:
# Используем лучший чекпоинт
save_path = "/data/nutrition/data/continued_best_model.pth"  # MAE=52.54

cfg = ContinueConfig()
cfg.EPOCHS = 50  # Ещё 50 эпох
cfg.HEAD_LR = 5e-5  # Ещё меньше LR

device = "cuda" if torch.cuda.is_available() else "cpu"

final_mae = train_resume_with_scheduler(cfg, device, save_path)
print(f"Финальный результат: {final_mae:.2f}")

In [ ]:
# Звпустим обучение ещё раз с момента, где остановились, с 50 эпохами и с тем же config
# Так как я просто запустила ячейку с тем же config, номера эпох не обновились, они снова будут с 200 до 249..
save_path = "/data/nutrition/data/continued_best_model.pth"  # MAE=50.97

cfg = ContinueConfig()
cfg.EPOCHS = 50  # Ещё 50 эпох
cfg.HEAD_LR = 5e-5  # Ещё меньше LR

device = "cuda" if torch.cuda.is_available() else "cpu"

final_mae = train_resume_with_scheduler(cfg, device, save_path)
print(f"Финальный результат: {final_mae:.2f}")

In [ ]:
print(f'Лучшая модель с результатом MAE=49.93 сохранена в файл {save_path}')

Этап 4. Валидация качества

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

test_tfms = get_transforms(cfg, split="test")
test_ds   = CaloriesDataset(cfg, test_tfms, split="test")
test_dl   = DataLoader(test_ds, batch_size=cfg.BATCH_SIZE, shuffle=False,
                       num_workers=cfg.NUM_WORKERS, pin_memory=True,
                       collate_fn=collate_fn)

# загружаем лучшую модель
model = MultimodalRegressor(cfg).to(device)
best_path = "/content/drive/MyDrive/nutrition/data/continued_best_model.pth"
state = torch.load(best_path, map_location=device)
model.load_state_dict(state)
model.eval()

# прогон на тестовых данных
all_preds, all_targets, dish_ids, img_paths = [], [], [], []

with torch.no_grad():
    for batch in test_dl:
        inputs = {
            "input_ids": batch["input_ids"].to(device),
            "attention_mask": batch["attention_mask"].to(device),
            "image": batch["image"].to(device),
            "mass": batch["mass"].to(device),
        }
        y = batch["target"].to(device)
        pred = model(**inputs)

        all_preds.extend(pred.cpu().numpy().tolist())
        all_targets.extend(y.cpu().numpy().tolist())

        dish_ids.extend(batch.get("dish_id", ["?"] * len(pred)))
        img_paths.extend(batch.get("image_path", [""] * len(pred)))

all_preds   = np.array(all_preds, dtype=float)
all_targets = np.array(all_targets, dtype=float)
mae = np.mean(np.abs(all_preds - all_targets))
print(f"Финальный тестовый MAE: {mae:.2f}")

In [ ]:
# собираем таблицу результатов
errors = np.abs(all_preds - all_targets)
res_df = pd.DataFrame({
    "dish_id": dish_ids,
    "image_path": img_paths,
    "y_true": all_targets,
    "y_pred": all_preds,
    "abs_error": errors,
})

worst5 = res_df.sort_values("abs_error", ascending=False).head(5)
display(worst5[["dish_id", "y_true", "y_pred", "abs_error"]])

# визуализация
n = len(worst5)
cols = 5
fig, axes = plt.subplots(1, n, figsize=(4*n, 4))
if n == 1:
    axes = [axes]

for ax, (_, row) in zip(axes, worst5.iterrows()):
    try:
        img = Image.open(row["image_path"]).convert("RGB")
        ax.imshow(img)
    except:
        ax.text(0.5, 0.5, "no image", ha="center", va="center")
    ax.axis("off")
    title = f"id:{row['dish_id']}\ntrue:{row['y_true']:.0f}  pred:{row['y_pred']:.0f}\nerr:{row['abs_error']:.0f}"
    ax.set_title(title, fontsize=10)
plt.tight_layout()
plt.show()

Выводы:

Большие ошибки наблюдаются на блюдах со смешанным составом (несколько компонентов с сильно различной калорийностью).

Модель недооценивает энергетическую плотность орехов и жареных компонентов.

Текстовые признаки («ингредиенты») могли быть сокращены или плохо токенизированы TinyBERT’ом, из-за чего текстовая часть модели не дала достаточного вклада.

На последних двух фотографиях вообще доствточно много ингредиентов смешано в одном блюде.

Финальный результат

Модель достигла MAE = 49.93 на тестовой выборке, что соответствует требуемому уровню точности (MAE < 50).
Проектная цель выполнена.